In [1]:
import os
import torch
import torch.utils.data
import pickle
import numpy as np
import random
import itertools
from tqdm import tqdm

In [17]:
class CustomDataPreprocessorForCNN():
    def __init__(self, input_seq_length=5, pred_seq_length=5, datasets=[i for i in range(37)], test_data_sets = [2], dev_ratio_to_test_set = 0.1, forcePreProcess=False):
        '''
        Initializer function for the CustomDataSetForCNN class
        params:
        input_seq_length : input sequence length to be considered
        output_seq_length : output sequence length to be predicted
        datasets : The indices of the datasets to use
        test_data_sets : The indices of the test sets from datasets
        dev_ratio_to_test_set : ratio of the validation set size to the test set size
        forcePreProcess : Flag to forcefully preprocess the data again from csv files
        '''
        # List of data directories where raw data resides
        self.data_dirs = ['./data/train/processed/biwi/biwi_hotel', './data/train/processed/crowds/arxiepiskopi1',
                          './data/train/processed/crowds/crowds_zara02', './data/train/processed/crowds/crowds_zara03',
                          './data/train/processed/crowds/students001', './data/train/processed/crowds/students003', 
                          './data/train/processed/stanford/bookstore_0',
                          './data/train/processed/stanford/bookstore_1', './data/train/processed/stanford/bookstore_2',
                          './data/train/processed/stanford/bookstore_3', './data/train/processed/stanford/coupa_3',
                          './data/train/processed/stanford/deathCircle_0', './data/train/processed/stanford/deathCircle_1',
                          './data/train/processed/stanford/deathCircle_2', './data/train/processed/stanford/deathCircle_3',
                          './data/train/processed/stanford/deathCircle_4', './data/train/processed/stanford/gates_0',
                          './data/train/processed/stanford/gates_1', './data/train/processed/stanford/gates_3',
                          './data/train/processed/stanford/gates_4', './data/train/processed/stanford/gates_5',
                          './data/train/processed/stanford/gates_6', './data/train/processed/stanford/gates_7',
                          './data/train/processed/stanford/gates_8', './data/train/processed/stanford/hyang_4',
                          './data/train/processed/stanford/hyang_5', './data/train/processed/stanford/hyang_6',
                          './data/train/processed/stanford/hyang_7', './data/train/processed/stanford/hyang_9',
                          './data/train/processed/stanford/nexus_0', './data/train/processed/stanford/nexus_1',
                          './data/train/processed/stanford/nexus_2', './data/train/processed/stanford/nexus_3',
                          './data/train/processed/stanford/nexus_4', './data/train/processed/stanford/nexus_7',
                          './data/train/processed/stanford/nexus_8', './data/train/processed/stanford/nexus_9']
        train_datasets = datasets
        for dataset in test_data_sets:
            train_datasets.remove(dataset)
        self.train_data_dirs = [self.data_dirs[x] for x in train_datasets]
        self.test_data_dirs = [self.data_dirs[x] for x in test_data_sets]
        
        # Number of datasets
        self.numDatasets = len(self.data_dirs)
        
        # Data directory where the pre-processed pickle file resides
        self.data_dir = './data/train/processed'
        
        # Store the arguments
        self.input_seq_length = input_seq_length
        self.pred_seq_length = pred_seq_length
        
        # Validation arguments
        self.dev_fraction = dev_ratio_to_test_set
        
        # Buffer for storing processed data.
        self.processed_input_output_pairs = []
        
        # Define the path in which the process data would be stored
        self.processed_train_data_file = os.path.join(self.data_dir, "trajectories_cnn_train.cpkl")
        self.processed_dev_data_file = os.path.join(self.data_dir, "trajectories_cnn_dev.cpkl")
        self.processed_test_data_file = os.path.join(self.data_dir, "trajectories_cnn_test.cpkl")
        
        # If the file doesn't exist or forcePreProcess is true
        if not(os.path.exists(self.processed_train_data_file)) or forcePreProcess:
            print("============ Creating pre-processed training data for CNN ============")
            self.preprocess(self.train_data_dirs, self.processed_train_data_file)
        if not(os.path.exists(self.processed_dev_data_file)) or not(os.path.exists(self.processed_test_data_file)) or forcePreProcess:
            print("============ Creating pre-processed dev & test data for CNN ============")
            self.preprocess(self.test_data_dirs, self.processed_test_data_file, self.dev_fraction, self.processed_dev_data_file)
        
    def preprocess(self, data_dirs, data_file, dev_fraction = 0., data_file_2 = None):
        random.seed(1) # Random seed for pedestrian permutation and data shuffling
        for directory in data_dirs:
            print('--> Processing dataset ' + str(directory))
            # define path of the csv file of the current dataset
            file_path = os.path.join(directory, 'world_pos_normalized.csv')
            
            # Load the data from the csv file
            data = np.genfromtxt(file_path, delimiter=',')
            
            # Frame IDs of the frames in the current dataset
            frameList = np.unique(data[0, :]).tolist()
            numFrames = len(frameList)
            
            # Frame ID increment for this dataset.
            frame_increment = np.min(np.array(frameList[1:-1]) - np.array(frameList[0:-2]))
            
            # For this dataset check which pedestrians exist in each frame.
            pedsInFrameList = []
            pedsPosInFrameList = []
            for ind, frame in enumerate(frameList):
                # For this frame check the pedestrian IDs.
                pedsInFrame = data[:, data[0, :] == frame]
                pedsList = pedsInFrame[1, :].tolist()
                pedsInFrameList.append(pedsList)
                # Position information for each pedestrian.
                pedsPos = []
                for ped in pedsList:
                    # Extract x and y positions
                    current_x = pedsInFrame[2, pedsInFrame[1, :] == ped][0]
                    current_y = pedsInFrame[3, pedsInFrame[1, :] == ped][0]
                    pedsPos.extend([current_x, current_y])
                    if (current_x == 0.0 and current_y == 0.0):
                        print('[WARNING] There exists a pedestrian at coordinate [0.0, 0.0]')
                pedsPosInFrameList.append(pedsPos)
            
            # Go over the frames in this data again to extract data.
            ind = 0
            
            while ind < len(frameList) - (self.input_seq_length + self.pred_seq_length):
                 # Check if this sequence contains consecutive frames. Otherwise skip this sequence.
                if not frameList[ind + self.input_seq_length + self.pred_seq_length] - frameList[ind] == (self.input_seq_length + self.pred_seq_length)*frame_increment:
                    ind += 1
                    continue;
                # List of pedestirans in this sequence.
                pedsList = np.unique(np.concatenate(pedsInFrameList[ind : ind + self.input_seq_length + self.pred_seq_length])).tolist()
                # Print the Frame numbers and pedestrian IDs in this sequence for sanity check.
                # print(str(int(self.input_seq_length + self.pred_seq_length)) + ' frames starting from Frame ' + str(int(frameList[ind])) +  ' contain pedestrians ' + str(pedsList))
                # Initialize numpy arrays for input-output pair
                data_input = np.zeros((2*len(pedsList), self.input_seq_length))
                data_output = np.zeros((2*len(pedsList), self.pred_seq_length))
                for ii in range(self.input_seq_length):
                    for jj in range(len(pedsList)):
                        if pedsList[jj] in pedsInFrameList[ind + ii]:
                            datum_index = pedsInFrameList[ind + ii].index(pedsList[jj])
                            data_input[2*jj:2*(jj + 1), ii] = np.array(pedsPosInFrameList[ind + ii][2*datum_index:2*(datum_index + 1)])
                for ii in range(self.pred_seq_length):
                    for jj in range(len(pedsList)):
                        if pedsList[jj] in pedsInFrameList[ind + self.input_seq_length + ii]:
                            datum_index = pedsInFrameList[ind + self.input_seq_length + ii].index(pedsList[jj])
                            data_output[2*jj:2*(jj + 1), ii] = np.array(pedsPosInFrameList[ind + self.input_seq_length + ii][2*datum_index:2*(datum_index + 1)])
                processed_pair = (torch.from_numpy(data_input), torch.from_numpy(data_output))
                self.processed_input_output_pairs.append(processed_pair)
                
                ind += self.input_seq_length + self.pred_seq_length
        print('--> Original Data Size: ' + str(len(self.processed_input_output_pairs)))   
        # Perform data augmentation
        self.augment_rotate()
        self.augment_flip()
        self.augment_permute()
            
        # Shuffle data, possibly divide into train and dev sets.
        print('--> Shuffling all data before saving')
        random.shuffle(self.processed_input_output_pairs)
        if dev_fraction != 0.:
            assert(data_file_2 != None)
            dev_size = int(len(self.processed_input_output_pairs)*dev_fraction)
            processed_dev_set = self.processed_input_output_pairs[:dev_size]
            processed_test_set = self.processed_input_output_pairs[dev_size:]
            # Save processed data.
            f = open(data_file, 'wb')
            print('--> Dumping test data to pickle file')
            pickle.dump(processed_test_set, f, protocol=2)
            f.close()
            f2 = open(data_file_2, 'wb')
            print('--> Dumping dev data to pickle file')
            pickle.dump(processed_dev_set, f2, protocol=2)
            f2.close()
            # Clear buffer
            self.processed_input_output_pairs = []
        else:
            # Save processed data.
            f = open(data_file, 'wb')
            print('--> Dumping training data to pickle file')
            pickle.dump(self.processed_input_output_pairs, f, protocol=2)
            f.close()
            # Clear buffer
            self.processed_input_output_pairs = []
    
    def augment_rotate(self):
        deg_increment_int = 30
        print('--> Data Augmentation 1: Coordinate Rotation (' + str(deg_increment_int) +' deg increment)')
        augmented_input_output_pairs = []
        for processed_input_output_pair in tqdm(self.processed_input_output_pairs):
            data_input, data_output = processed_input_output_pair[0].numpy(), processed_input_output_pair[1].numpy()
            num_peds = int(data_input.shape[0]/2)
            # Rotate by deg_increment deg sequentially
            for deg in range(deg_increment_int, 360, deg_increment_int):
                data_input_rotated = np.zeros_like(data_input)
                data_output_rotated = np.zeros_like(data_output)
                rad = np.radians(deg)
                c, s = np.cos(rad), np.sin(rad)
                Rot = np.array(((c,-s), (s, c)))
                for ii in range(num_peds):
                    for jj in range(self.input_seq_length):
                        coordinates_for_this_ped = data_input[2*ii:2*(ii+1), jj]
                        new_coordinates_for_this_ped = np.dot(Rot, coordinates_for_this_ped)
                        data_input_rotated[2*ii:2*(ii+1), jj] = new_coordinates_for_this_ped
                    for jj in range(self.pred_seq_length):
                        coordinates_for_this_ped = data_output[2*ii:2*(ii+1), jj]
                        new_coordinates_for_this_ped = np.dot(Rot, coordinates_for_this_ped)
                        data_output_rotated[2*ii:2*(ii+1), jj] = new_coordinates_for_this_ped
                processed_pair_rotated = (torch.from_numpy(data_input_rotated), torch.from_numpy(data_output_rotated))
                augmented_input_output_pairs.append(processed_pair_rotated)
        self.processed_input_output_pairs.extend(augmented_input_output_pairs)
        print('--> Augmented Data Size: ' + str(len(self.processed_input_output_pairs)))
    
    def augment_flip(self):
        print('--> Data Augmentation 2: Y Flip')
        augmented_input_output_pairs = []
        for processed_input_output_pair in tqdm(self.processed_input_output_pairs):
            data_input, data_output = processed_input_output_pair[0].numpy(), processed_input_output_pair[1].numpy()
            num_peds = int(data_input.shape[0]/2)
            # Flip y
            data_input_yflipped = np.zeros_like(data_input)
            data_output_yflipped = np.zeros_like(data_output)
            for kk in range(num_peds):
                data_input_yflipped[2*kk, :] = data_input[2*kk, :]
                data_input_yflipped[2*kk+1, :] = -1*data_input[2*kk+1, :]
                data_output_yflipped[2*kk, :] = data_output[2*kk, :]
                data_output_yflipped[2*kk+1, :] = -1*data_output[2*kk+1, :]
            processed_pair_yflipped = (torch.from_numpy(data_input_yflipped), torch.from_numpy(data_output_yflipped))
            augmented_input_output_pairs.append(processed_pair_yflipped)
        self.processed_input_output_pairs.extend(augmented_input_output_pairs)
        print('--> Augmented Data Size: ' + str(len(self.processed_input_output_pairs)))
        
    def augment_permute(self):
        # Specify how many pedestrian permutations to consider per input-output pair
        num_perms = 4
        print('--> Data Augmentation 3: Pedestrian Permutation (' + str(num_perms) + ' random permutations per input-output pair)')
        augmented_input_output_pairs = []
        for processed_input_output_pair in tqdm(self.processed_input_output_pairs):
            data_input, data_output = processed_input_output_pair[0].numpy(), processed_input_output_pair[1].numpy()
            num_peds = int(data_input.shape[0]/2)
            for ii in range(num_perms):
                perm = np.random.permutation(num_peds)
                data_input_permuted = np.zeros_like(data_input)
                data_output_permuted = np.zeros_like(data_output)
                for jj in range(len(perm)):
                    data_input_permuted[2*jj:2*(jj+1), :] = data_input[2*perm[jj]:2*(perm[jj]+1), :]
                    data_output_permuted[2*jj:2*(jj+1), :] = data_output[2*perm[jj]:2*(perm[jj]+1), :]
                processed_pair_permuted = (torch.from_numpy(data_input_permuted), torch.from_numpy(data_output_permuted))
                augmented_input_output_pairs.append(processed_pair_permuted)
        self.processed_input_output_pairs.extend(augmented_input_output_pairs)
        print('--> Augmented Data Size: ' + str(len(self.processed_input_output_pairs)))
                    

In [18]:
processed = CustomDataPreprocessorForCNN(forcePreProcess=True, test_data_sets=[2], dev_ratio_to_test_set=0.3)

============ Creating pre-processed training data for CNN ============
--> Processing dataset ./data/train/processed/biwi/biwi_hotel
--> Processing dataset ./data/train/processed/crowds/arxiepiskopi1
--> Processing dataset ./data/train/processed/crowds/crowds_zara03
--> Processing dataset ./data/train/processed/crowds/students001
--> Processing dataset ./data/train/processed/crowds/students003
--> Processing dataset ./data/train/processed/stanford/bookstore_0
--> Processing dataset ./data/train/processed/stanford/bookstore_1
--> Processing dataset ./data/train/processed/stanford/bookstore_2
--> Processing dataset ./data/train/processed/stanford/bookstore_3
--> Processing dataset ./data/train/processed/stanford/coupa_3
--> Processing dataset ./data/train/processed/stanford/deathCircle_0
--> Processing dataset ./data/train/processed/stanford/deathCircle_1
--> Processing dataset ./data/train/processed/stanford/deathCircle_2
--> Processing dataset ./data/train/processed/stanford/deathCircl



  0%|          | 0/2273 [00:00<?, ?it/s]

--> Processing dataset ./data/train/processed/stanford/nexus_9
--> Original Data Size: 2273
--> Data Augmentation 1: Coordinate Rotation (30 deg increment)




  4%|▍         | 99/2273 [00:00<00:02, 985.23it/s]

  7%|▋         | 154/2273 [00:00<00:02, 785.04it/s]

  8%|▊         | 191/2273 [00:00<00:06, 314.52it/s]

 10%|▉         | 221/2273 [00:00<00:08, 249.35it/s]

 11%|█         | 249/2273 [00:00<00:07, 257.29it/s]

 13%|█▎        | 286/2273 [00:00<00:07, 282.31it/s]

 14%|█▍        | 318/2273 [00:00<00:06, 289.60it/s]

 16%|█▌        | 353/2273 [00:01<00:06, 305.28it/s]

 18%|█▊        | 409/2273 [00:01<00:05, 352.95it/s]

 20%|██        | 464/2273 [00:01<00:04, 393.52it/s]

 23%|██▎       | 525/2273 [00:01<00:03, 439.53it/s]

 26%|██▌       | 589/2273 [00:01<00:03, 484.23it/s]

 29%|██▉       | 658/2273 [00:01<00:03, 529.70it/s]

 32%|███▏      | 719/2273 [00:01<00:02, 548.82it/s]

 34%|███▍      | 778/2273 [00:01<00:03, 493.25it/s]

 37%|███▋      | 831/2273 [00:01<00:03, 459.63it/s]

 39%|███▊      | 880/2273 [00:02<00:03, 422.28it/s]

 41%|████      | 935/2273 [00:02<00:02, 451.86it/s]

 43%|████▎     | 983/2273 [00:02<00:02, 431.3

--> Augmented Data Size: 27276
--> Data Augmentation 2: Y Flip


 14%|█▍        | 3948/27276 [00:00<00:01, 17848.05it/s]

 18%|█▊        | 4986/27276 [00:00<00:02, 11030.56it/s]

 23%|██▎       | 6193/27276 [00:00<00:01, 11322.10it/s]

 30%|██▉       | 8068/27276 [00:00<00:01, 12848.64it/s]

 37%|███▋      | 10010/27276 [00:00<00:01, 14299.67it/s]

 42%|████▏     | 11473/27276 [00:00<00:01, 14064.20it/s]

 48%|████▊     | 12958/27276 [00:00<00:01, 14290.72it/s]

 53%|█████▎    | 14405/27276 [00:00<00:00, 13822.45it/s]

 61%|██████    | 16672/27276 [00:01<00:00, 15654.75it/s]

 71%|███████   | 19253/27276 [00:01<00:00, 17748.75it/s]

 79%|███████▉  | 21637/27276 [00:01<00:00, 19220.09it/s]

 87%|████████▋ | 23707/27276 [00:01<00:00, 17537.88it/s]

 94%|█████████▍| 25652/27276 [00:01<00:00, 18070.72it/s]

100%|██████████| 27276/27276 [00:01<00:00, 16901.83it/s]

  0%|          | 0/54552 [00:00<?, ?it/s]

  0%|          | 187/54552 [00:00<00:29, 1847.72it/s]

--> Augmented Data Size: 54552
--> Data Augmentation 3: Pedestrian Permutation (10 random permutations per input-output pair)




  0%|          | 270/54552 [00:00<00:47, 1148.04it/s]

  1%|          | 465/54552 [00:00<00:41, 1308.34it/s]

  1%|▏         | 724/54552 [00:00<00:35, 1535.70it/s]

  2%|▏         | 914/54552 [00:00<00:32, 1628.78it/s]

  2%|▏         | 1106/54552 [00:00<00:31, 1706.13it/s]

  3%|▎         | 1389/54552 [00:00<00:27, 1936.04it/s]

  3%|▎         | 1670/54552 [00:00<00:24, 2135.14it/s]

  3%|▎         | 1897/54552 [00:00<00:24, 2164.24it/s]

  4%|▍         | 2134/54552 [00:01<00:23, 2220.77it/s]

  5%|▍         | 2468/54552 [00:01<00:21, 2467.71it/s]

  5%|▌         | 2810/54552 [00:01<00:19, 2692.62it/s]

  6%|▌         | 3095/54552 [00:01<00:18, 2725.05it/s]

  6%|▌         | 3379/54552 [00:01<00:18, 2701.27it/s]

  7%|▋         | 3720/54552 [00:01<00:17, 2879.76it/s]

  7%|▋         | 4017/54552 [00:01<00:23, 2160.55it/s]

  8%|▊         | 4266/54552 [00:02<00:39, 1285.10it/s]

  8%|▊         | 4460/54552 [00:02<00:44, 1127.74it/s]

  8%|▊         | 4622/54552 [00:02<00:50, 995.79it

KeyboardInterrupt: 

In [ ]:
train_file = open(processed.processed_train_data_file, 'rb')
dev_file = open(processed.processed_dev_data_file, 'rb')
test_file = open(processed.processed_test_data_file, 'rb')

In [157]:
processed.processed_train_data_file

'./data/train/processed/trajectories_cnn_train.cpkl'

In [158]:
train = pickle.load(train_file)
dev = pickle.load(dev_file)
test = pickle.load(test_file)

In [159]:
len(train)

2273

In [160]:
len(dev)

30

In [161]:
len(test)

72

In [16]:
class CustomDatasetForCNN(torch.utils.data.Dataset):
    def __init__(self, file_path):
        self.file_path = file_path
        self.file = open(self.file_path, 'rb')
        self.data = pickle.load(self.file)
        self.file.close()
    
    def __getitem__(self, index):
        item = self.data[index]
        return item
    
    def __len__(self):
        return len(self.data)     
        

In [17]:
train_set = CustomDatasetForCNN(processed.processed_train_data_file)

In [18]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=1, shuffle=True)

In [23]:
x, y = train_set.__getitem__(99)

In [24]:
x

tensor([[0.1771, 0.1612, 0.1471, 0.1313, 0.1158],
        [0.6733, 0.6807, 0.6873, 0.6947, 0.6982]], dtype=torch.float64)

In [25]:
next(iter(train_loader))

[tensor([[[ 0.1777,  0.1773,  0.1773,  0.1773,  0.1773],
          [ 0.3633,  0.3771,  0.3907,  0.4044,  0.4180],
          [ 0.1191,  0.1191,  0.1191,  0.1191,  0.1191],
          [ 0.0447,  0.0447,  0.0447,  0.0447,  0.0447],
          [ 0.0983,  0.0983,  0.0983,  0.0983,  0.0983],
          [ 0.0544,  0.0544,  0.0544,  0.0544,  0.0544],
          [ 0.0721,  0.0721,  0.0721,  0.0721,  0.0721],
          [ 0.0598,  0.0598,  0.0598,  0.0598,  0.0598],
          [ 0.0661,  0.0661,  0.0661,  0.0661,  0.0661],
          [ 0.0401,  0.0401,  0.0401,  0.0401,  0.0401],
          [-0.3741, -0.3772, -0.3802, -0.3820, -0.3850],
          [ 0.7355,  0.7404,  0.7452,  0.7528,  0.7576],
          [ 0.1136,  0.1219,  0.1327,  0.1418,  0.1506],
          [ 0.2656,  0.2787,  0.2928,  0.3077,  0.3223],
          [-0.1503, -0.1503, -0.1503, -0.1503, -0.1503],
          [ 0.2184,  0.2184,  0.2184,  0.2184,  0.2184],
          [ 0.1071,  0.1047,  0.1019,  0.0973,  0.0945],
          [ 0.1414,  0.1318,  0

In [22]:
x, y = train_set.__getitem__(9)

In [26]:
len(train_loader)

56592